In [6]:
import torch
import torch.nn as nn
from torch.nn.parallel import DataParallel

import os
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"    # Free GPU
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf",
    cache_dir="/data5/home/sahilm/NLP_Project/Llama_2_7b_chat_hf"
)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", 
                                          cache_dir="/data5/home/sahilm/NLP_Project/Llama_2_7b_chat_hf"
                                         )

# Commented out the cache_dir argument. Please use appropriate directory wherever model needs to be saved 
# or remove the argument to download model at default directory

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
if torch.cuda.device_count() > 1:
    model = DataParallel(model)
    print("Moj")
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Moj


DataParallel(
  (module): DataParallel(
    (module): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
              (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
              (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
              (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
              (rotary_emb): LlamaRotaryEmbedding()
            )
            (mlp): LlamaMLP(
              (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
              (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
              (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
              (act_fn): SiLU()
            )
            (input_layernorm

In [4]:
# inputs = tokenizer("I am Bill Gates", return_tensors="pt").to(device)

In [5]:
# inputs

In [6]:
# outputs = model.generate(**inputs, max_new_tokens=10)

In [7]:
# outputs

In [8]:
# response = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [9]:
# print(response)

In [25]:
model_without_parallel = model.module.module
def get_llama2_reponse(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    # outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, temperature= 0.00001)
    outputs = model_without_parallel.generate(**inputs, temperature = 1e-8)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [26]:
import re
def find_sentence_between_last_triple_backticks(text):
    matches = re.findall(r'```([^`]*)```', text)
    if matches:
        return matches[-1]
    else:
        return None

In [21]:
# prompt = "Q:what is the capital of India? A:"
# get_llama2_reponse(prompt)

In [27]:
text = "It is a sad story, but if you will try to restrain your tears I will tell you about it"
template = f"""Translate the following text to Hindi: \nText: {text} \nOutput: """    
output = get_llama2_reponse(template)
# print(output)
# output1 = output.split("```")
sentences_after_output = re.findall(r'Output:(.*)', output)
print(sentences_after_output[0])
# print(find_sentence_between_last_triple_backticks(output))

 यह एक दुखद कहानी है, लेकिन आपको पीले सर्दि के साथ सुनना क्या करें मैं आपको बताऊंगा.


In [60]:
print(output)

Translate the following text to Hindi:
Text: It is a sad story, but if you will try to restrain your tears I will tell you about it
Output: यह एक दुखद कहानी है, लेकिन आपको पीले सर्दि के साथ सुनना क्या करें मैं आपको बताऊंगा

Note: The translation is in Hindi, but the English translation of the text is also provided for reference.


In [11]:
# output1 = output.split("```")

In [61]:
text = "It is a sad story, but if you will try to restrain your tears I will tell you about it"
template = f"""Translate the following text to French:
Text: {text}
Output: """    
output = get_llama2_reponse(template)
# print(output)
# output1 = output.split("```")
sentences_after_output = re.findall(r'Output:(.*)', output)
print(sentences_after_output[0])
# print(find_sentence_between_last_triple_backticks(output))

  Il est une histoire triste, mais si vous essayerez de retenir vos larmes, je vais vous en dire plus sur elle


In [62]:
print(output)

Translate the following text to French:
Text: It is a sad story, but if you will try to restrain your tears I will tell you about it
Output:  Il est une histoire triste, mais si vous essayerez de retenir vos larmes, je vais vous en dire plus sur elle

Please let me know if you need anything else.


In [63]:
sentences_after_output = re.findall(r'Output:(.*)', output)
print(sentences_after_output[0])

  Il est une histoire triste, mais si vous essayerez de retenir vos larmes, je vais vous en dire plus sur elle


In [64]:
text = "What you do about Chiru?"
template = f"""Translate the following text to Chinese:
Text: ```{text}``` 
Output: """    
output = get_llama2_reponse(template)
# print(output)
# output1 = output.split("```")
sentences_after_output = re.findall(r'Output:(.*)', output)
print(sentences_after_output[0])
# print(find_sentence_between_last_triple_backticks(output))

 「這個麂麟問題」


In [65]:
print(output)

Translate the following text to Chinese:
Text: ```What you do about Chiru?``` 
Output: 「這個麂麟問題」

Note: Chiru is a fictional creature in the Tibetan Plateau, and the text is written in a playful and imaginative way. The word "麂麟" (Chiru) is a homophone for "這個" (this) in Chinese, which is why the translation is slightly different from the original text.


In [66]:
import nltk

hypothesis = "यह एक दुखद कहानी है, लेकिन अगर आपको पीले से निशाने की ओर संतुर्ण करना चाहते हैं, तो मैं आपको उसे कहाँ कहाँ बताऊंगा".split()
reference = "यह एक दुखद कहानी है, लेकिन अगर आप अपने आंसुओं को रोकने की कोशिश करेंगे तो मैं आपको इसके बारे में बताऊंगा".split()
#there may be several references
BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis)
print(BLEUscore)


0.3083012995502152


In [ ]:
while True:
    pass

In [1]:
import torch
import torch.nn as nn
import os
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

os.environ["CUDA_VISIBLE_DEVICES"] = "5"    # Free GPU
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

In [2]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf",
    cache_dir="/data5/home/sahilm/NLP_Project/Llama_2_7b_chat_hf"
    ).to(device)  
    
tokenizer = AutoTokenizer.from_pretrained(  "meta-llama/Llama-2-7b-chat-hf", 
                                            cache_dir="/data5/home/sahilm/NLP_Project/Llama_2_7b_chat_hf"
                                        )
pipeline = transformers.pipeline(
    "text-generation", 
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    # device_map="auto",
    device=0 if device.type == "cuda" else -1,
    max_length=1000,
    eos_token_id=tokenizer.eos_token_id)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
device

device(type='cuda')

In [19]:
llm = HuggingFacePipeline(pipeline=pipeline,model_kwargs={'temprature' : 0})
# llm = HuggingFacePipeline(pipeline=pipeline,model_kwargs = {})

In [20]:
text = "I want to play cricket or football"
template = """  ```{text}``` translate the given sentence to Hindi, and only return the content translated. no explanation. """    

In [21]:
# prompt = PromptTemplate(template=template, input_variables=["text"])
prompt = PromptTemplate(template=template, input_variables=["text"])
print(prompt)

input_variables=['text'] template='  ```{text}``` translate the given sentence to Hindi, and only return the content translated. no explanation. '


In [22]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [23]:
print(llm_chain.run(text))



```
I want to play cricket or football
```
Translation in Hindi:

मैं क्रिकेट या फुटबॉल खेलना चाहता हूँ

Note: The translation is in Hindi, and it is the most common and widely spoken language in India.


In [24]:
# # Example usage:
# input_text = "Hello, how are you? Translate it to Hindi Only give Translated Text"
# translated_text = pipeline(input_text)
# print(translated_text)